<h1 style='color:blue;'> Clasificación de texto </h1> 

Importar librerias de interés

In [1]:
import numpy as np
import pandas as pd

import re 
import string
import nltk


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

porter = PorterStemmer()
lancaster = LancasterStemmer()
wordnet_lemma = WordNetLemmatizer()

nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cx001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cx001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cx001\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


leer los archivos

In [2]:
train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('comentarios.csv',encoding='utf-16', sep=',',  error_bad_lines=False)

b'Skipping line 41116: expected 7 fields, saw 11\nSkipping line 41247: expected 7 fields, saw 11\nSkipping line 67200: expected 7 fields, saw 11\nSkipping line 69160: expected 7 fields, saw 11\n'


In [3]:
train.rename(columns={'identity_hate':'identity_attack'}, inplace=True)
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_attack
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
test.rename(columns={'comment':'comment_text'}, inplace=True)
test.head()

,id,comment_text,creation_time,source,tweet_id,user,user_id
0,0,RT @NoSurrenderHK: @guardiannews The truth is ...,2020-06-13 16:07:02,Twitter for Android,1271836487663808513,currentecalamo,1005852785609326592
1,1,RT @MarisaKabas: Content warning: police bruta...,2020-06-13 16:07:02,Twitter for Android,1271836487663775744,Donald Dire,1088300096666591232
2,2,RT @shahmiruk: It is absolutely unfair &amp; d...,2020-06-13 16:07:02,Twitter for iPhone,1271836487693275144,tanya cochrane 🕷#FBPE,25872176
3,3,RT @autotheoryqueen: TERFs police the boundari...,2020-06-13 16:07:02,Twitter for iPhone,1271836487747862529,Michael Bermingham,59031350
4,4,looks like a scary demon witch monster to me,2020-06-13 16:07:02,Twitter for iPhone,1271836487676579841,untossable chum,2771192143


Preparación de texto: se remueven signos de puntuación y se convierte todo el texto en minuscula

In [5]:
def clean_text_round1(text):
    '''make text lowercase, remove punctuation.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[\n]', ' ', text)
    text = re.sub('@\w+', ' ', text)
    text = re.sub('rt', ' ', text)
    return text

round1 = lambda x: clean_text_round1(str(x))

In [6]:
test.comment_text = test.comment_text.apply(round1)
train.comment_text = train.comment_text.apply(round1)

In [7]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_attack
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [8]:
test.head()

,id,comment_text,creation_time,source,tweet_id,user,user_id
0,0,nosurrenderhk guardiannews the truth is hk n...,2020-06-13 16:07:02,Twitter for Android,1271836487663808513,currentecalamo,1005852785609326592
1,1,marisakabas content warning police brutality...,2020-06-13 16:07:02,Twitter for Android,1271836487663775744,Donald Dire,1088300096666591232
2,2,shahmiruk it is absolutely unfair amp dising...,2020-06-13 16:07:02,Twitter for iPhone,1271836487693275144,tanya cochrane 🕷#FBPE,25872176
3,3,autotheoryqueen terfs police the boundaries ...,2020-06-13 16:07:02,Twitter for iPhone,1271836487747862529,Michael Bermingham,59031350
4,4,looks like a scary demon witch monster to me,2020-06-13 16:07:02,Twitter for iPhone,1271836487676579841,untossable chum,2771192143


### Implementar modelo de clasificación

Este codigo se construyo usando la referencia: https://www.kaggle.com/thousandvoices/logistic-regression-with-words-and-char-n-grams

In [9]:
#Nombrar las clases
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_attack']

In [10]:
df_train_subset = train.loc[:,class_names]
df_train_text = train.loc[:, 'comment_text']
df_train_subset.head()

,toxic,severe_toxic,obscene,threat,insult,identity_attack
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train_text, df_train_subset, test_size= 0.3, random_state=13)

#### TF-IDF

In [12]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features=30000)
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=30000)
vectorizer = make_union(word_vectorizer, char_vectorizer, n_jobs=2)

In [13]:
word_vectorizer.fit(X_train)
X_train_word_features = word_vectorizer.transform(X_train)

In [14]:
test_features = word_vectorizer.transform(X_test)
word_features = word_vectorizer.transform(test['comment_text'])

#### Generar Scores

In [15]:
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
import sklearn.metrics as metrics
from tqdm import tqdm
import pickle

submission = pd.DataFrame.from_dict({'id': test['id']})
df_classification_report = pd.DataFrame(columns=['Class_Name','Log_loss', 'Accuracy', 'Accuracy_vs_Test'])
losses = []
auc = []
auctest= []

dict_confussion_matrix = {} # TODO ADD ELEMENTS TO DICT
cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_attack']

for class_name in tqdm(cols):
    train_target = y_train[class_name]
    test_target  = y_test[class_name]
    
    
    clf = LogisticRegression(solver='sag')
    
    cv_loss = np.mean(cross_val_score(clf, X_train_word_features, train_target, scoring='roc_auc'))
    losses.append(cv_loss)

    clf.fit(X_train_word_features, train_target)
    
    cv_score = precision_score(train_target, clf.predict(X_train_word_features))
    auc.append(cv_score)
    

    y_pred = clf.predict(test_features)
    submission[class_name] = clf.predict_proba(word_features)[:, 1]
    auc_score = metrics.roc_auc_score(test_target, y_pred)
    auctest.append(auc_score) 
    
    # TODO EXPORT CLASSIFIER AS PICKLE OBJECT ALSO THE TFIDF
    pickle.dump( clf, open( "LogisticRegression"+class_name+".pkl", "wb" ) )
    #plot confusion matrix
    confusion_matrix(test_target, y_pred)
    

df_classification_report['Class_Name'] = cols
df_classification_report['Log_loss'] = losses
df_classification_report['Accuracy'] = auc
df_classification_report['Accuracy_vs_Test'] = auctest


100%|██████████| 6/6 [00:40<00:00,  6.78s/it]


In [16]:
df_classification_report

,Class_Name,Log_loss,Accuracy,Accuracy_vs_Test
0,toxic,0.969129,0.942029,0.793841
1,severe_toxic,0.983446,0.634199,0.610257
2,obscene,0.982496,0.936645,0.798781
3,threat,0.985455,0.824561,0.570396
4,insult,0.975673,0.863213,0.749949
5,identity_attack,0.971404,0.771654,0.573629


In [17]:
submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_attack
0,0,0.043557,0.005468,0.020624,0.001808,0.021551,0.006116
1,1,0.018151,0.006819,0.017256,0.001763,0.015761,0.005264
2,2,0.648313,0.007552,0.104505,0.001872,0.202844,0.072936
3,3,0.038006,0.002571,0.024342,0.000929,0.016490,0.009064
4,4,0.115772,0.005776,0.029941,0.002434,0.028826,0.007462
...,...,...,...,...,...,...,...
88981,89995,0.157962,0.011046,0.073745,0.002551,0.028364,0.007351
88982,89996,0.081629,0.006220,0.023299,0.005346,0.032134,0.012904
88983,89997,0.100021,0.005401,0.018202,0.004139,0.028385,0.009318
88984,89998,0.029168,0.006640,0.024732,0.002034,0.017069,0.005632


In [18]:
submission = pd.merge(test, submission, on='id')
submission.head()

,id,comment_text,creation_time,source,tweet_id,user,user_id,toxic,severe_toxic,obscene,threat,insult,identity_attack
0,0,nosurrenderhk guardiannews the truth is hk n...,2020-06-13 16:07:02,Twitter for Android,1271836487663808513,currentecalamo,1005852785609326592,0.043557,0.005468,0.020624,0.001808,0.021551,0.006116
1,1,marisakabas content warning police brutality...,2020-06-13 16:07:02,Twitter for Android,1271836487663775744,Donald Dire,1088300096666591232,0.018151,0.006819,0.017256,0.001763,0.015761,0.005264
2,2,shahmiruk it is absolutely unfair amp dising...,2020-06-13 16:07:02,Twitter for iPhone,1271836487693275144,tanya cochrane 🕷#FBPE,25872176,0.648313,0.007552,0.104505,0.001872,0.202844,0.072936
3,3,autotheoryqueen terfs police the boundaries ...,2020-06-13 16:07:02,Twitter for iPhone,1271836487747862529,Michael Bermingham,59031350,0.038006,0.002571,0.024342,0.000929,0.016490,0.009064
4,4,looks like a scary demon witch monster to me,2020-06-13 16:07:02,Twitter for iPhone,1271836487676579841,untossable chum,2771192143,0.115772,0.005776,0.029941,0.002434,0.028826,0.007462


Cargar los datos de salida a un CSV

In [19]:
submission.to_csv('submission.csv', index=False)